In [1]:
# Example of how to fit an LCMS model
####################################################################################################

try:
  import geeViz.getImagesLib as getImagesLib
except:
  !python -m pip install geeViz
  import geeViz.getImagesLib as getImagesLib


#Module imports
# import geeViz.getImagesLib as getImagesLib
import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
ee = getImagesLib.ee

#Set up to mapper objects to use
#Can use the default one first
Map = getImagesLib.Map

# Enter proxy url (see in url bar- e.g. https://someCode-dot-someRegion.notebooks.googleusercontent.com/)
# If left as None, will prompt user at first Map.view() call 
Map.proxy_url = 'https://3a96040caeb4cbe7-dot-us-west3.notebooks.googleusercontent.com/'
print('done')

Initializing GEE
Reauthentication is needed. Please run `gcloud auth application-default login` to reauthenticate.


Enter verification code:  4/1AZEOvhVKT_wkWjkJXPC_u1A9OQgET8J0tqL31T2rujEQ4CUZE2KZxvF5TgE



Successfully saved authorization token.
Successfully initialized
geeViz package folder: /opt/conda/lib/python3.10/site-packages/geeViz
done


In [81]:
# projects/lcms-292214/assets/R8/PR_USVI/Training-Tables/Training-Tables_AnnualizedFormat_v2022
# projects/lcms-292214/assets/R8/PR_USVI/Training-Tables/Training-Tables_AnnualizedFormat_v2022
Map.clearMap()
Map.port = 1234

# Set the projection
crs = getImagesLib.common_projections['NLCD_CONUS']['crs']
transform  = getImagesLib.common_projections['NLCD_CONUS']['transform']
scale = None

# Bring in annualized TimeSync data
# Each plot has one feature for each year from 2000 to 2020
# This results in over 20k plots
timeSyncData = ee.FeatureCollection('projects/lcms-292214/assets/R8/PR_USVI/TimeSync/18_PRVI_AllPlots_TimeSync_Annualized_Table_secLC')
training_years = [int(yr) for yr in timeSyncData.aggregate_histogram('YEAR').keys().getInfo()]
plot_ids = timeSyncData.aggregate_histogram('PLOTID').keys().getInfo()
print('Training years:',training_years)
print('Unique Plot ID Count:',len(plot_ids))
print('Total Training Plots:', timeSyncData.size().getInfo())



Training years: [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
Unique Plot ID Count: 1000
Total Training Plots: 20992


In [82]:
# Let's view the TimeSync data
Map.addLayer(timeSyncData.map(lambda f:f.buffer(15).bounds(5,crs)),{'layerType':'geeVectorImage'},'LCMS PRUSVI TimeSync Training Data')
Map.turnOnInspector()
Map.view()

Adding layer: LCMS PRUSVI TimeSync Training Data
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1234/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3a96040caeb4cbe7-dot-us-west3.notebooks.googleusercontent.com/proxy/1234/geeView/?accessToken=None


127.0.0.1 - - [31/Jul/2023 21:52:22] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


In [67]:
# Now we will bring in LandTrendr data for each year

landTrendrCollection = ee.ImageCollection('projects/rcr-gee/assets/landTrendr-lcms-training-module-2')

landTrendrCollectionInfo = landTrendrCollection.first().toDictionary().getInfo()

Map.clearMap()

# View exported LT output
# Convert stacked outputs into collection of fitted, magnitude, slope, duration, etc values for each year
lt_fit = changeDetectionLib.batchSimpleLTFit(landTrendrCollection,landTrendrCollectionInfo['startYear'],landTrendrCollectionInfo['endYear'],None,bandPropertyName='band',arrayMode=True)

# Vizualize image collection for charting (opacity set to 0 so it will chart but not be visible)
Map.addLayer(lt_fit,{'opacity':0},'LT Fit TS')

# Visualize fitted landTrendr composites
fitted_bns = lt_fit.select(['.*_fitted']).first().bandNames()
out_bns = fitted_bns.map(lambda bn: ee.String(bn).split('_').get(0))

# Give same names as composites
lt_synth = lt_fit.select(fitted_bns,out_bns)

# Visualize  LandTrendr fitted 
Map.addTimeLapse(lt_synth,getImagesLib.vizParamsFalse,'Synthetic Composite Timelapse')


# # Join the raw and fited values
# ltJoined = getImagesLib.joinCollections(composites.select(bandNames),lt_fit.select(['.*_fitted']))
# print(ltJoined.first().bandNames().getInfo())
# Map.addLayer(ltJoined,{'min':0.2,'max':1},'Raw and LT Fitted',True)


Map.turnOnInspector()
Map.view()

Adding layer: LT Fit TS
Adding layer: Synthetic Composite Timelapse
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1234/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3a96040caeb4cbe7-dot-us-west3.notebooks.googleusercontent.com/proxy/1234/geeView/?accessToken=None


127.0.0.1 - - [31/Jul/2023 21:45:55] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


In [37]:
# And then bring in CCDC data for each year
Map.clearMap()

ccdcTiles = ee.ImageCollection('projects/rcr-gee/assets/tiledOutputs-lcms-training-module-4')
composites = ee.ImageCollection('projects/rcr-gee/assets/composites-lcms-training-module-1')
ccdcInfo = ccdcTiles.first().toDictionary().getInfo()
startYear = 1999
endYear = 2023
startJulian = 1
endJulian = 365

ccdcImg = ccdcTiles.mosaic()
#Specify which harmonics to use when predicting the CCDC model
#CCDC exports the first 3 harmonics (1 cycle/yr, 2 cycles/yr, and 3 cycles/yr)
#If you only want to see yearly patterns, specify [1]
#If you would like a tighter fit in the predicted value, include the second or third harmonic as well [1,2,3]
whichHarmonics = [1,2,3]

#Whether to fill gaps between segments' end year and the subsequent start year to the break date
fillGaps = False


annualSegCoeffs = changeDetectionLib.annualizeCCDC(ccdcImg, startYear, endYear, startJulian, endJulian, tEndExtrapolationPeriod = 0.3, 
  yearStartMonth = 4, yearStartDay = 1, annualizeWithCompositeDates = False, compositeCollection = composites)
annualCCDC = changeDetectionLib.getFitSlopeCCDC(annualSegCoeffs, startYear, endYear)
print(annualCCDC.first().bandNames().getInfo())

# Synthetic composites visualizing
# Take common false color composite bands and visualize them for the next to the last year

# First get the bands of predicted bands and then split off the name
fittedBns = annualCCDC.select(['.*_predicted']).first().bandNames()
bns = fittedBns.map(lambda bn: ee.String(bn).split('_').get(0))

# Filter down to the next to the last year and a summer date range
syntheticComposites = annualCCDC.select(fittedBns,bns)\
    .filter(ee.Filter.calendarRange(endYear-1,endYear-1,'year'))\
    .first()

# Visualize output as you would a composite
Map.addLayer(syntheticComposites,getImagesLib.vizParamsFalse,'Synthetic Composite')

Map.turnOnInspector()
Map.view()


Adding layer: time
['year', 'blue_CCDC_INTP', 'blue_CCDC_SLP', 'blue_CCDC_COS1', 'blue_CCDC_SIN1', 'blue_CCDC_COS2', 'blue_CCDC_SIN2', 'blue_CCDC_COS3', 'blue_CCDC_SIN3', 'green_CCDC_INTP', 'green_CCDC_SLP', 'green_CCDC_COS1', 'green_CCDC_SIN1', 'green_CCDC_COS2', 'green_CCDC_SIN2', 'green_CCDC_COS3', 'green_CCDC_SIN3', 'red_CCDC_INTP', 'red_CCDC_SLP', 'red_CCDC_COS1', 'red_CCDC_SIN1', 'red_CCDC_COS2', 'red_CCDC_SIN2', 'red_CCDC_COS3', 'red_CCDC_SIN3', 'nir_CCDC_INTP', 'nir_CCDC_SLP', 'nir_CCDC_COS1', 'nir_CCDC_SIN1', 'nir_CCDC_COS2', 'nir_CCDC_SIN2', 'nir_CCDC_COS3', 'nir_CCDC_SIN3', 'swir1_CCDC_INTP', 'swir1_CCDC_SLP', 'swir1_CCDC_COS1', 'swir1_CCDC_SIN1', 'swir1_CCDC_COS2', 'swir1_CCDC_SIN2', 'swir1_CCDC_COS3', 'swir1_CCDC_SIN3', 'swir2_CCDC_INTP', 'swir2_CCDC_SLP', 'swir2_CCDC_COS1', 'swir2_CCDC_SIN1', 'swir2_CCDC_COS2', 'swir2_CCDC_SIN2', 'swir2_CCDC_COS3', 'swir2_CCDC_SIN3', 'NDVI_CCDC_INTP', 'NDVI_CCDC_SLP', 'NDVI_CCDC_COS1', 'NDVI_CCDC_SIN1', 'NDVI_CCDC_COS2', 'NDVI_CCDC_SIN2',

127.0.0.1 - - [31/Jul/2023 21:16:42] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


In [59]:
# LCMS also uses various elevation/terrain variables to help our models
Map.clearMap()
terrainStack = []

# Can use any elevation data
# Will use SRTM for this example
elevation = ee.Image("USGS/SRTMGL1_003").resample('bicubic')
terrain = ee.Algorithms.Terrain(elevation)

terrainStack.append(terrain)

# A very simple algorithm for elevation position
# Good at finding ridges and depressions
def tpi(elevation,n):
    return elevation.subtract(elevation.focalMean(n)).rename([f'TPI_{int(n*2)}'])

# A nice palette from: https://github.com/gee-community/ee-palettes
# Use cmocean.speed[7]
palette = ['fffdcd', 'e1cd73', 'aaac20', '5f920c', '187328', '144b2a', '172313']

Map.addLayer(elevation.reproject(crs,transform),{'min':0,'max':250,'palette':palette},'Elevation')

# Get TPI for 11 and 21 pixel diameter circular kernels
for r in [5.5,10.5]:
    tpiR = tpi(elevation,r)
    terrainStack.append(tpiR)
    Map.addLayer(tpiR.reproject(crs,transform),{'min':-10,'max':10,'palette':palette},f'TPI {int(r*2)}')

# Stack all terrain data
terrainStack = ee.Image.cat(terrainStack)
print('Available terrain bands:',terrainStack.bandNames().getInfo())
Map.view()

Adding layer: Elevation
Adding layer: TPI 11
Adding layer: TPI 21
Available terrain bands: ['elevation', 'slope', 'aspect', 'hillshade', 'TPI_11', 'TPI_21']
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1234/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://3a96040caeb4cbe7-dot-us-west3.notebooks.googleusercontent.com/proxy/1234/geeView/?accessToken=None


127.0.0.1 - - [31/Jul/2023 21:39:12] "GET /geeView/js/runGeeViz.js HTTP/1.1" 200 -


In [86]:
# Now extract LandTrendr, CCDC, and terrain values for each TimeSync plot from its respective year

#Provide location composites will be exported to
#This should be an asset folder, or more ideally, an asset imageCollection
exportPathRoot = 'projects/rcr-gee/assets/timeSync-extractions-lcms-training-module-4'
# Make sure collection exists
aml.create_image_collection(exportPathRoot)

# Currently geeView within Colab uses a different project to authenticate through, so you may need to make your asset public to view from within Colab
aml.updateACL(exportPathRoot,writers = [],all_users_can_read = True,readers = [])


# Extract for each year
for training_year in training_years[11:12]:
    
    # Filter TimeSync data for the given year
    timeSyncData_yr = timeSyncData.filter(ee.Filter.eq('YEAR',training_year))
   
    # Filter the LandTrendr and CCDC data for the given year
    lt_yr = lt_fit.filter(ee.Filter.calendarRange(training_year,training_year,'year')).first()
    ccdc_yr = annualCCDC.filter(ee.Filter.calendarRange(training_year,training_year,'year')).first()

    # Get a stack of all predictor bands
    extractionStack = ee.Image.cat([lt_yr,ccdc_yr,terrainStack])

    # Extract the values
    extracted_values = extractionStack.reduceRegions(collection=timeSyncData_yr, reducer = ee.Reducer.first(), scale = None, crs = crs, crsTransform = transform, tileScale = 4)
    
    # Export them as an asset
    assetName = f'LCMS_Training_TimeSync_yr{training_year}_LandTrendr_CCDC_Terrain_Extraction'
    assetPath = f'{exportPathRoot}/assetName'
    print(assetName,assetPath)
    t = ee.batch.Export.table.toAsset(collection = extracted_values,
                      description = assetName,
                      assetId = assetPath)
    print('Exporting:',assetName)
    print(t)
    # t.start()


Collection projects/rcr-gee/assets/timeSync-extractions-lcms-training-module-4 already exists
Updating permissions for:  projects/rcr-gee/assets/timeSync-extractions-lcms-training-module-4
[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]
2011 1000
LCMS_Training_TimeSync_yr2011_LandTrendr_CCDC_Terrain_Extraction projects/rcr-gee/assets/timeSync-extractions-lcms-training-module-4/assetName
Exporting: LCMS_Training_TimeSync_yr2011_LandTrendr_CCDC_Terrain_Extraction
<Task EXPORT_FEATURES: LCMS_Training_TimeSync_yr2011_LandTrendr_CCDC_Terrain_Extraction (UNSUBMITTED)>
